In [9]:
from aocd import get_data

Each report is a list of numbers called levels that are separated by spaces.

In the example above, the reports can be found safe or unsafe by checking those rules:

- `7 6 4 2 1`: Safe because the levels are all decreasing by 1 or 2.
- `1 2 7 8 9`: Unsafe because 2 7 is an increase of 5.
- `9 7 6 2 1`: Unsafe because 6 2 is a decrease of 4.
- `1 3 2 4 5`: Unsafe because 1 3 is increasing but 3 2 is decreasing.
- `8 6 4 4 1`: Unsafe because 4 4 is neither an increase or a decrease.
- `1 3 6 7 9`: Safe because the levels are all increasing by 1, 2, or 3.
So, in this example, 2 reports are safe.

Analyze the unusual data from the engineers. How many reports are safe?

In [1]:
import numpy as np

def is_unsafe(diffs):
    """
    Unused function from when I was messing around but I'm leaving it here
    """
    increasing_too_much = np.any(diffs >= 4)
    decreasing_too_much = np.any(diffs <= -4)
    consecutive_same = np.any(diffs == 0)
    increase_then_decrease = np.any((diffs[:-1] > 0) & (diffs[1:] < 0))
    
    return (
        increasing_too_much or 
        decreasing_too_much or 
        consecutive_same or 
        increase_then_decrease
    )
    

def is_safe(diffs):
    all_decreasing_safely = np.all((diffs == -1) | (diffs == -2) | (diffs == -3))
    all_increasing_safely = np.all((diffs == 1) | (diffs == 2) | (diffs == 3))

    return all_decreasing_safely or all_increasing_safely


def assess(arr):
    # diffs between consecutive elements
    diffs = np.diff(arr)

    return is_safe(diffs)

Check against the examples given

In [2]:
a = (7, 6, 4, 2, 1)
b = (1, 2, 7, 8, 9)
c = (9, 7, 6, 2, 1)
d = (1, 3, 2, 4, 5)
e = (8, 6, 4, 4, 1)
f = (1, 3, 6, 7, 9)

In [22]:
print(assess(a))
print(assess(b))
print(assess(c))
print(assess(d))
print(assess(e))
print(assess(f))

True
False
False
False
False
True


In [10]:
def count_true(arr):
    arr = np.asarray(arr)
    return np.sum(arr == True)

We can't process this in a vectorized manner as the lines have variable lengths :(

In [23]:
num_safe = 0
for line in get_data(day=2, year=2024).splitlines():
    array = np.fromstring(line, dtype=int, sep=" ")
    num_safe += assess(array)

In [24]:
count_true(results)

np.int64(639)

The Problem Dampener is a reactor-mounted module that lets the reactor safety systems tolerate a single bad level in what would otherwise be a safe report. It's like the bad level never happened!

Now, the same rules apply as before, except if removing a single level from an unsafe report would make it safe, the report instead counts as safe.

More of the above example's reports are now safe:

- `7 6 4 2 1`: Safe without removing any level.
- `1 2 7 8 9`: Unsafe regardless of which level is removed.
- `9 7 6 2 1`: Unsafe regardless of which level is removed.
- `1 3 2 4 5`: Safe by removing the second level, 3.
- `8 6 4 4 1`: Safe by removing the third level, 4.
- `1 3 6 7 9`: Safe without removing any level.

Thanks to the Problem Dampener, 4 reports are actually safe!

Update your analysis by handling situations where the Problem Dampener can remove a single level from unsafe reports. How many reports are now safe?

If an unsafe level change is found, drop it from the array and assess again.

In [13]:
def reassess(arr):
    diffs = np.diff(arr)
    return is_safe_with_dampening(diffs)
    
def is_safe_with_dampening(diffs, num_unsafe_allowed=1):
    num_safe_decreasing = np.sum((diffs == -1) | (diffs == -2) | (diffs == -3))
    num_safe_increasing = np.sum((diffs == 1) | (diffs == 2) | (diffs == 3))

    faulty_entries = diffs.size - max(num_safe_decreasing, num_safe_increasing)

    return faulty_entries <= num_unsafe_allowed


In [17]:
reassess(b)

np.True_

In [18]:
b

(1, 2, 7, 8, 9)

In [20]:
bdiffs = np.diff(b)
b_safe_increasing = np.sum((bdiffs == 1) | (bdiffs == 2) | (bdiffs == 3))
b_safe_decreasing = np.sum((bdiffs == -1) | (bdiffs == -2) | (bdiffs == -3))

bdiffs.size - max(b_safe_decreasing, b_safe_increasing)

np.int64(1)

In [21]:
b_safe_increasing

np.int64(3)

In [14]:
results = []
for line in get_data(day=2, year=2024).splitlines():
    array = np.fromstring(line, dtype=int, sep=" ")
    safe = reassess(array)

    results.append(safe)

np.sum(results)

np.int64(678)

In [15]:
def load_data_as_arrays():
    data = get_data(day=2, year=2024)
    return [np.fromstring(line, dtype=int, sep=" ") for line in data.splitlines()]

def assess_all(arrays, num_unsafe_allowed=1):
    # compute diffs for all rows at once
    diffs = np.diff(arrays, axis=1)

    # filters to apply all checks at once
    safe_decreasing = (diffs >= -3) & (diffs <= -1)
    safe_increasing = (diffs >= 1) & (diffs <= 3)
    
    faulty_entries = diffs.shape[1] - np.maximum(np.sum(safe_decreasing, axis=1), np.sum(safe_increasing, axis=1))
    
    return faulty_entries <= num_unsafe_allowed


arrays = load_data_as_arrays()
results = np.array([assess(arr) for arr in arrays])

# count True results
count_safe = np.sum(results)

In [16]:
count_safe

np.int64(639)

Urgh... wrong again. I'll come back to this